<a href="https://colab.research.google.com/github/BBlopusy/machinelearningstudies/blob/main/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KNN

In [ ]:
import pandas as pd


## Pobranie danych i ich przygotowanie
**!wget** to linuksowe polecenie ściągające plik z sieci

In [ ]:
!wget https://github.com/betacord/ML/raw/master/lab1/data/data.csv

### Wczytanie danych 

In [ ]:
houses = pd.read_csv('/content/data.csv')

In [ ]:
houses

,AppraisedValue,SqFtLot,WaterSystem,SewerSystem,Access,Topography,StreetSurface,RestrictiveSzShape,InadequateParking,PcntUnusable,...,NativeGrowthProtEsmt,Easements,OtherDesignation,DeedRestrictions,DevelopmentRightsPurch,WaterProblems,TranspConcurrency,OtherProblems,lat,long
0,619000.0,6542712,1,1,4,0,1,0,0,0,...,0,0,0,0,1,0,0,0,47.354019,-122.225166
1,708000.0,105850,2,1,3,0,2,0,0,0,...,0,0,0,0,0,0,0,0,47.611248,-122.053154
2,297000.0,1667,2,2,4,0,1,0,2,0,...,0,0,0,0,0,0,0,0,47.701248,-122.365784
3,523000.0,1872,2,2,4,0,1,0,2,0,...,0,0,0,0,0,0,0,0,47.537918,-122.392845
4,288000.0,3480,2,2,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,47.510227,-122.182167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52415,523000.0,6982,2,2,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,47.769810,-122.367447
52416,425000.0,5481,2,2,4,0,1,0,2,0,...,0,0,0,0,0,0,0,0,47.510895,-122.366936
52417,377000.0,6793,2,2,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,47.766289,-122.358192
52418,581000.0,7200,2,2,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,47.769989,-122.364822


Wyświetlenie kolumn i wczytanie ich w czystej postaci tablicy numphy:

In [ ]:
houses.columns.values

array(['AppraisedValue', 'SqFtLot', 'WaterSystem', 'SewerSystem',
       'Access', 'Topography', 'StreetSurface', 'RestrictiveSzShape',
       'InadequateParking', 'PcntUnusable', 'MtRainier', 'Olympics',
       'Cascades', 'Territorial', 'SeattleSkyline', 'PugetSound',
       'LakeWashington', 'LakeSammamish', 'SmallLakeRiverCreek',
       'OtherView', 'WfntLocation', 'WfntFootage', 'WfntBank',
       'WfntPoorQuality', 'WfntRestrictedAccess', 'WfntAccessRights',
       'WfntProximityInfluence', 'TidelandShoreland', 'LotDepthFactor',
       'TrafficNoise', 'PowerLines', 'OtherNuisances', 'NbrBldgSites',
       'Contamination', 'AdjacentGolfFairway', 'AdjacentGreenbelt',
       'HistoricSite', 'CurrentUseDesignation', 'NativeGrowthProtEsmt',
       'Easements', 'OtherDesignation', 'DeedRestrictions',
       'DevelopmentRightsPurch', 'WaterProblems', 'TranspConcurrency',
       'OtherProblems', 'lat', 'long'], dtype=object)

### Selekcja atrybutów

Selekcja atrybutów, wybieramy te które nas interesują ⛱ 
Czyli wartość, wielkość działki, system nawadniający itd.
My wybierzemy dokładnie za radą ekspertów: wielkość działki i współrzędne geograficzne. Nie wpływają w sposób liniowy. Będziemy próbować przewidzieć wartość czyli AppraisedValue na podstawie SqFtlot i współrzednych geograficznych czyli lat i long. 


In [ ]:
# wyodrębnienie atrybutu decyzyjnego
values = houses['AppraisedValue']
# usuwamy go z systemu decyjnego, tworząc w ten sposób system informacyjny
houses.drop('AppraisedValue',1, inplace=True) 
#wskazujemy czy chcemy usunąć po osi wierszy czy po osi kolumn, my wskazujemy że po osi kolumn
#dlatego dajemy w parametrze axis wartość: 1
#nie chcemy zwróconej kopi tej ramki tylko żeby została zastąpiona oryginalna parametrem inplance =True


In [ ]:
houses

,SqFtLot,WaterSystem,SewerSystem,Access,Topography,StreetSurface,RestrictiveSzShape,InadequateParking,PcntUnusable,MtRainier,...,NativeGrowthProtEsmt,Easements,OtherDesignation,DeedRestrictions,DevelopmentRightsPurch,WaterProblems,TranspConcurrency,OtherProblems,lat,long
0,6542712,1,1,4,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,47.354019,-122.225166
1,105850,2,1,3,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,47.611248,-122.053154
2,1667,2,2,4,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,47.701248,-122.365784
3,1872,2,2,4,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,47.537918,-122.392845
4,3480,2,2,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,47.510227,-122.182167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52415,6982,2,2,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,47.769810,-122.367447
52416,5481,2,2,4,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,47.510895,-122.366936
52417,6793,2,2,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,47.766289,-122.358192
52418,7200,2,2,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,47.769989,-122.364822


### **Normalizacja danych**
Mamy różne atrybuty i różne wartości np. w SqFtLot mamy duże i małe wartości, rozsrtrzelone mocno. Musimy przetransformować dane, pozmieniać wartości żeby różnice były mniejsze, żeby rozkłady prawdopodobieństwa do siebie się upodobniły ale jednocześnie, żeby zależności miedzy nimi dalej były te same.

In [ ]:
houses = (houses - houses.mean()) / (houses.max() - houses.min()) 
#pomnijszamy przez wartość średnią i dzielimy przez różnicę wartości maksymalną i minimalną w całej ramce danych
houses = houses[['lat', 'long', 'SqFtLot']] #zostawiamy sobie te trzy atrybuty które nas interesują

In [ ]:
houses

,lat,long,SqFtLot
0,-0.002260,-0.002427,0.926695
1,0.004289,-0.001374,0.012250
2,0.006580,-0.003288,-0.002550
3,0.002422,-0.003454,-0.002521
4,0.001717,-0.002164,-0.002293
...,...,...,...
52415,0.008325,-0.003298,-0.001795
52416,0.001734,-0.003295,-0.002008
52417,0.008236,-0.003241,-0.001822
52418,0.008330,-0.003282,-0.001764


### Podział na trenig i test
Trening 80% test 20% 
20:41 długość filmu ✌